In [1]:
import torch

if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
device = torch.device(dev)

/home/cairne/anaconda3/envs/mol_torch_gcnn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## make donor model

In [1]:
import sys

sys.path.append("/home/cairne/PythonProj/SmartChemDesign/mol_torch_model/Source")

from Source.trainer import ModelTrainer
from Source.model import MolGraphNet
from Source.data import train_test_valid_split
from Source.mol_featurizer import featurize_sdf
from torch_geometric.loader import DataLoader

path_to_sdf = "/home/cairne/PythonProj/SmartChemDesign/mol_torch_model/Data/logS_logP.sdf"
valuenames = ["logP"]
output_path = "Output"
n_split = 5
output_mark = f"logP_data_{valuenames[0]}"

featurized_train = featurize_sdf(path_to_sdf, valuenames)
folds, test_loader = train_test_valid_split(featurized_train, n_split, test_ratio=0.1, batch_size=2000)

trainer = ModelTrainer(MolGraphNet, folds, test_loader, output_path, output_mark, mode="regression")
trainer.train_cv_models()


/home/cairne/anaconda3/envs/mol_torch_gcnn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/cairne/anaconda3/envs/mol_torch_gcnn/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1584: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(
Missing logger folder: Output/logP_data_logP_regression_2022_08_16_12_47_45/fold_1/logs/default

  | Name            | Type              | Params
------------------------------------------------------
0 | conv_actf       | LeakyReLU         | 0     
1 | linear_actf     | LeakyReLU         | 0     
2 | conv_sequential | Sequential_7afd7d | 544 K 
3 | lin_sequential 

Validation sanity check:   0%|          | 0/1 [00:00<?, ?it/s]

/home/cairne/anaconda3/envs/mol_torch_gcnn/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


RuntimeError: mat1 and mat2 shapes cannot be multiplied (0x75 and 128x64)

## make acceptor model

In [1]:
import sys

sys.path.append("/home/cairne/PythonProj/SmartChemDesign/mol_torch_model/Source")

from Source.trainer import ModelTrainer
from Source.model import MolGraphNet
from Source.data import train_test_valid_split
from Source.mol_featurizer import featurize_sdf
from torch_geometric.loader import DataLoader

path_to_sdf = "/home/cairne/PythonProj/SmartChemDesign/mol_torch_model/Data/Am_ML_logS_logP_train.sdf"
test_sdf = "/home/cairne/PythonProj/SmartChemDesign/mol_torch_model/Data/Am_ML_logS_logP_test.sdf"
valuenames = ["logS"]
output_path = "Output/AD_check_models"
n_split = 5
output_mark = f"Am_data_{valuenames[0]}"

featurized_train = featurize_sdf(path_to_sdf, valuenames)
folds = train_test_valid_split(featurized_train, n_split, test_ratio=0, batch_size=1, subsample_size=False, return_test=False)

featurized_test = featurize_sdf(test_sdf, valuenames)
test_loader = DataLoader([val for i, val in enumerate(featurized_test)], batch_size=1)

trainer = ModelTrainer(MolGraphNet, folds, test_loader, output_path, output_mark, mode="regression")
trainer.train_cv_models(pretrained_folder="/home/cairne/PythonProj/SmartChemDesign/mol_torch_model/Output/AD_check_models/Results_Eu_data_logP_regression_2022_08_15_22_48_47",
                        layers_to_freeze=0)


/home/cairne/anaconda3/envs/mol_torch_gcnn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Error(s) in loading state_dict for MolGraphNet:
	size mismatch for conv_sequential.module_0.module_0.lins_l.0.weight: copying a param with shape torch.Size([256, 75]) from checkpoint, the shape in current model is torch.Size([64, 128]).
	size mismatch for conv_sequential.module_0.module_0.lins_l.0.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for conv_sequential.module_0.module_0.lins_l.1.weight: copying a param with shape torch.Size([256, 75]) from checkpoint, the shape in current model is torch.Size([64, 128]).
	size mismatch for conv_sequential.module_0.module_0.lins_l.1.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for conv_sequential.module_0.module_0.lins_l.2.weight: copying a param with shape torch.Size([256, 75]) from checkpoint, the shape in current model is torch.Size([64, 128]).
	size mismatch 

## Make metals models for donor/acceptor pairs

In [ ]:
import sys

sys.path.append("/home/cairne/PythonProj/SmartChemDesign/mol_torch_model/Source")

from Source.trainer import ModelTrainer
from Source.model import MolGraphNet
from Source.data import train_test_valid_split
from Source.mol_featurizer import featurize_sdf
from torch_geometric.loader import DataLoader

metals = ["Al", "Cu", "Cd", "Mg", "Pb", "Ni", "Zn"]
valuenames = ["logK"]
output_path = "/home/cairne/PythonProj/SmartChemDesign/mol_torch_model/Output/add_metals_for_AD"
n_split = 5
for metal in metals:
    output_mark = f"{metal}_data_{valuenames[0]}"
    path_to_sdf = f"/home/cairne/PythonProj/SmartChemDesign/mol_torch_model/Data/metals_sdf_csv/converted/{metal}_ML.sdf"

    featurized_train = featurize_sdf(path_to_sdf, valuenames)
    folds, test_loader = train_test_valid_split(featurized_train, n_split, test_ratio=0.2, batch_size=30)

    trainer = ModelTrainer(MolGraphNet, folds, test_loader, output_path, output_mark, mode="regression")
    trainer.train_cv_models()
